In [ ]:
from twikit import Client
from twikit import TwitterException 
from twikit import TooManyRequests
from twikit.utils import Endpoint
from translate import Translator
from math import ceil
import time
import json
import requests
import random

In [ ]:
# this API requires authentication
f = open('authentication.txt', 'r')
auth = f.read()
f.close()
auth_token = auth.split("\n")

# don't hardcode your email and password into something!!!
# the auth is in gitignore so I won't get hacked
USERNAME = str(auth_token[0])
EMAIL = str(auth_token[1])
PASSWORD = str(auth_token[2])

# Initialize client
client = Client(language='en-US', http2=True)

# Login to the service with provided user credentials
client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

In [ ]:
# Twitter LOVES to ban people when they log in repeatedly
# saving the cookies makes sure I don't get banned (often)

client.get_cookies()
client.save_cookies('IGNOREcookies.json')
with open('IGNOREcookies.json', 'r', encoding='UTF8') as f:
    client.set_cookies(json.load(f))

In [ ]:
# housekeeping function
# each different method uses a different API endpoint
# each different API endpoint has a rate limit
# you can hit it a certain number of times per a time period (usually 15 minutes)
# this tells me how much time I have left if I've hit the rate limit

def get_limit_reset_time(endpoint: str):
    res = requests.get(
        endpoint,
        headers=client._base_headers,
        cookies=client.get_cookies()
    )
    return ceil(int(res.headers['x-rate-limit-reset']) - time.time())

In [ ]:
# timeout check for scraping tweet IDs
try:
    print(client.search_tweet(
        f'from:JoeBiden since:2020-01-01 until:2021-03-01', 'Latest', count=40
    ))
except TooManyRequests:
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

In [ ]:
# timeout check for processing tweets
try:
    print(client.get_tweet_by_id(1351951465674276869))
except TooManyRequests:
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

In [ ]:
# another housekeeping function
# if I'm suddenly getting 403 errors, I can use this to check if I've been banned
# sometimes I just have to go on the browser and reauthenticate

def check_user_status(user_id):
    """
    True if the user is active, otherwise false (not exists or suspended).
    """
    try:
        client.get_user_by_id(user_id)
    except TwitterException as e:
        if str(e).startswith('Invalid user id'):
            return False
        raise e
    else:
        return True

check_user_status(1547081484695216130)

In [ ]:
# INPUT: the user handle, a beginning and end of a date range
# OUTPUT: the user handle, the user_ID, and the scraped tweets
def get_all_tweets(handle, since, until):

    # load the cookies so you don't login a million times and get banned
    client.load_cookies('IGNOREcookies.json')

    # initialize the list we will store our data in
    mass_tweets = []

    since = f'{since}-01-01'
    until = f'{until}-12-31'

    # this will pull the first forty tweets
    tweets = client.search_tweet(
        f'from:{handle} since:{since} until:{until}', 'Top'
    )
    tweets1 = [tweet.id for tweet in tweets]
    mass_tweets += tweets1

    # if it returns an empty list, the user had no available tweets during the date time range
    if len(tweets) == 0:
        return([])
    
    # this will keep looking for tweets until a certain number of them has been reached
    while len(tweets) > 0 and len(mass_tweets) < 30:

        # this API provides a 'tweet' object, but we only want the id when we return
        tweets = tweets.next()
        tweets1 = [tweet.id for tweet in tweets]
        mass_tweets += tweets1

        # keep pulling tweets until number is hit or there are none left

        # we need to make a check in case we've hit the max number of tweets we can scrape
        # this prevents us from pinging the API for no reason
        if len(tweets) == 0:
            print("No more tweets")
            break
        else:
            print(f'Total tweets = {len(mass_tweets)}')
            continue

    return(mass_tweets)

In [ ]:
def process_tweets(handle, user_id, name, tweet_ids, since):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('IGNOREcookies.json')

    # initialize a list to store all tuples
    tweets = []

    i = 0
    for tweet_id in tweet_ids:
        try:
            # using the IDs we pulled from above
            tweet = client.get_tweet_by_id(tweet_id)

            # we have international data
            # this will translate it and identify it's translation
            if tweet.lang != 'en':
                translator = Translator(to_lang='en')
                tweet_text = translator.translate(tweet.text)
                tweets.append([int(tweet.id), int(user_id), name, handle, str(tweet_text), str(tweet.lang), 'True', 'en', str(tweet.created_at_datetime), since])

            # otherwise we just move on
            else:
                tweets.append((int(tweet.id), int(user_id), name, handle, str(tweet.text), str(tweet.lang), 'False', 'null', str(tweet.created_at_datetime), since))
        
            print(f'Tweet {i} processed')
            i += 1

        # it throws an Index Error if the tweet has been deleted/ is not available
        except IndexError:
            print(f'Index Error: {tweet}')
        
        time.sleep(.5)

    return tweets

In [ ]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.execute(""" SELECT twitter_user_id, politician_name, twitter_handle, election_year
            FROM coordinates
            WHERE twitter_active_during_election = 'True'
            """)
active_user_list = c.fetchall()
print(active_user_list[0])

In [ ]:
user_list = active_user_list.copy()

In [ ]:
print(user_list[0])

In [ ]:
final_tweet_list = []
no_tweets_list = []

while len(user_list) > 0:

    tweet_id_list = []
    i = 0

    # this first loop pulls tweet IDs for politicians
    while (i < 5 and len(user_list) > 0):
        
        # we're only ever concerned with the first value
        # we pop this later to keep the list moving
        lst = user_list[0]

        # all of our parameters for the functions
        user_id = lst[0]
        name = lst[1]
        handle = lst[2]
        since = lst[3]
        until = lst[3]+1

        # pull tweet IDs that we will user in second while loop
        tweet_ids = get_all_tweets(handle, since=since, until=until)
        print(f'{len(tweet_ids)} tweets collected for {name} for {since} election')

        # pop user so we're not in an infinite loop
        user_list.pop(0)

        # we only add to the counter if that person actually had tweets to process
        if len(tweet_ids) > 0:
            i+=1
            tweet_id_list.append([handle, user_id, name, tweet_ids, since])
        
        # I want to keep track of which politicians didn't tweet during their election year
        else:
            no_tweets_list.append([user_id, name, handle, since])
        
    tweet_id_list_copy = tweet_id_list.copy()
    tweet_list = []

    # this second loop processes and cleans the tweets
    # it formats each tweet into a list that can be executed into the SQL database
    while len(tweet_id_list_copy) > 0:
        
        # same thing here, we're only ever concerned with index 0
        # these are lists of lists
        index = tweet_id_list_copy[0]

        # all the parameters we need
        handle = index[0]
        user_id = index[1]
        name = index[2]
        year = index[4]
        
        # we only want to do up to 25 tweets per politician
        # this means we can process 5 politicians per rate timeout
        # 150 tweets per 15 minutes
        # not great but it's free

        if len(index[3]) > 25:

            # random sample in an attempt to stay unbiased
            tweet_ids = random.sample(index[3], 25)
        else:
            tweet_ids = index[3]
        
        # process the tweets and add them to our holding list from above
        tweets = process_tweets(handle, user_id, name, tweet_ids, year)
        tweet_list.append(tweets)
        
        # pop the first index to keep the loop going
        tweet_id_list_copy.pop(0)
    
    # once we've run both loops, we add to the final list
    # this way we don't lose the data once it restarts
    final_tweet_list += tweet_list

    # rate limit protection
    # we can only process 150 tweets per 15 minutes
    # getting errors from rate limits throws off the whole thing
    # this prevents that from happening

In [ ]:
for item in final_tweet_list[0]:
    print(type(item))

In [ ]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.execute("""CREATE TABLE tweets (
          tweet_id INTEGER,
          user_id INTEGER,
          user_name STRING,
          user_handle STRING,
          tweet_text STRING,
          tweet_original_lang STRING,
          tweet_translated STRING,
          tweet_translated_lang STRING,
          created_date STRING,
          election_year INT
          )""")

In [ ]:
conn.execute('DROP TABLE tweets')

In [ ]:
# upload data in SQLite table
# storing this data is important since this is an unofficial API
# every time I access it, I am risking not being able to access it again

for lst in final_tweet_list:
    for tweet in lst:
        print(tweet)
        c.execute("INSERT INTO tweets VALUES (?,?,?,?,?,?,?,?,?,?)", tweet)
        conn.commit()
